# 1.2 Fingerprint hashing

Using the previously selected data with the features you found pertinent, you have to:

Implement your minhash function from scratch. No ready-made hash functions are allowed. Read the class material and search the internet if you need to. For reference, it may be practical to look at the description of hash functions in the book.

Process the dataset and add each record to the MinHash. The subtask's goal is to try and map each consumer to its bin; to ensure this works well, be sure you understand how MinHash works and choose a matching threshold to use. Before moving on, experiment with different thresholds, explaining your choice.

In [6]:
import pandas as pd
from tqdm import tqdm as tq
import warnings
import numpy as np
warnings.filterwarnings("ignore")

In [7]:
df = pd.read_csv("data/data.csv", sep = '\t')

### SAVE GENDER AS STRING IN PART 1 INSTEAD OF CONVERTING IT HERE

In [8]:
df.CustGender = df.CustGender.astype(str)

In [32]:
df

,Unnamed: 0,TransactionID,CustGender,CustomerClassAge,Richness,Expenditure
0,0,T1,0,age_1,richness_6,exp_1
1,1,T2,1,age_4,richness_2,exp_10
2,2,T3,0,age_1,richness_6,exp_6
3,3,T4,0,age_3,richness_10,exp_9
4,4,T5,0,age_2,richness_4,exp_9
...,...,...,...,...,...,...
1041139,1041139,T1048563,1,age_2,richness_4,exp_7
1041140,1041140,T1048564,1,age_1,richness_7,exp_6
1041141,1041141,T1048565,1,age_2,richness_10,exp_7
1041142,1041142,T1048566,1,age_3,richness_4,exp_7


In [9]:
del df['Unnamed: 0']

# 1.2.1 Shingles

First of all we build the shingles from all the unique values per column in the loaded dataset. We ignore the `TransactionID` column because it is not a shingle.

In [98]:
shingles = []
for column_name in df.columns[1:]:
    shingles += sorted(list(df[column_name].unique()))

In [99]:
shingles

['0',
 '1',
 'age_0',
 'age_1',
 'age_2',
 'age_3',
 'age_4',
 'age_5',
 'age_6',
 'richness_1',
 'richness_10',
 'richness_2',
 'richness_3',
 'richness_4',
 'richness_5',
 'richness_6',
 'richness_7',
 'richness_8',
 'richness_9',
 'exp_1',
 'exp_10',
 'exp_2',
 'exp_3',
 'exp_4',
 'exp_5',
 'exp_6',
 'exp_7',
 'exp_8',
 'exp_9']

# 1.2.2 Create Shingle Matrix

First of all we create the function which maps each transaction into a vector of 0/1 based on the shingles. 

In [11]:
def one_hot_vector(data, index):
    """Creates a one hot vector for the row found in the data at the given index based on the shingles.
    
    :args
    data - a pandas dataframe containing the data.
    index - an int which corresponds to the row that will be turned into a one hot vector.
    
    :returns
    a numpy array one hot representation of the row
    """
    
    values = data.loc[index][['CustGender', 'CustomerClassAge', 'Richness', 'Expenditure']].values
    
    indeces = np.where(values.reshape(values.size, 1) == vocabulary)[1]
    vector = np.zeros(len(shingles))
    vector[indeces] = 1
    
    return vector

Example: 

In [38]:
print(one_hot_vector(df, 1))

[0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]


Now we can build a sparse matrix with all the encoded transiction: 

In [29]:
shingle_matrix = np.zeros((1041144, 28))

for i in tq(range(len(df))):
    # Append the one hot vectors as rows
    shingle_matrix[df.index[i]] = one_hot_vector(df, i) 

# We need to transpose because for the shuffling, the Shingles need to be the rows.
shingle_matrix = shingle_matrix.T

100%|██████████████████████████████████████████████████████████████████████| 1041144/1041144 [13:10<00:00, 1317.41it/s]


In [30]:
%store shingle_matrix

Stored 'boolean_matrix' (ndarray)


# 1.2.3 Create the Signature Matrix
From the Shingle Matrix, we will now create the signature matrix by doing the following:
1. Shuffle the rows of the Shingle Matrix.
1. Create a vector where each element corresponds to the index of the row of each column (Shingle) where the first 1 is found.
1. Append this vector to the Signature Matrix.
1. Repeat $n$ times.

In [79]:
n_permutations = 12
signature_matrix = np.zeros((12, shingle_matrix.shape[1]))

In [80]:
for i in tq(range(n_permutations)):
    # 1. Shuffle rows
    np.random.shuffle(shingle_matrix)
    
    # 2. Create the vector of indeces where the first 1 is found. np.argmax stops at the first occurrence
    signature_row = np.argmax(shingle_matrix == 1, axis=0) + 1
    
    # 3. Add to signature matrix
    signature_matrix[i] = signature_row

100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:12<00:00,  1.05s/it]


# 1.2.4 Divide into Signature Matrix into Bands

The example signature matrix below is divided into $b$ bands of $r$ rows each, and each band is hashed separately. For this example, we are setting band , which means that we will consider any titles with the same first two rows to be similar. The larger we make b the less likely there will be another Paper that matches all of the same permutations.

![signature_matrix_into_bands](https://storage.googleapis.com/lds-media/images/locality-sensitive-hashing-lsh-buckets.width-1200.png)

Ultimately, the size of the bands control the probability that two items with a given Jaccard similarity end up in the same bucket. If the number of bands is larger, you will end up with much smaller sets. For instance, $b = p$, where $p$ is the number of permutations (i.e. rows in the signature matrix) would almost certainly lead to $N$ buckets of only one item because there would be only one item that was perfect similar across every permutation.

In [101]:
b = 2

How to put the items into buckets?
* create $p \times \frac{b}{p}$ chunks of the signature matrix such that each chunk has $b$ rows.
* 